<a href="https://colab.research.google.com/github/BeArnab96/MCP_Prediction/blob/master/GA_arnab_concised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
import zipfile, io, os

def upload_dir_file(case_f):
    # author: yasser mustafa, 21 March 2018  
    # case_f = 0 for uploading one File or Package(.py) and case_f = 1 for uploading one Zipped Directory
    uploaded = files.upload()    # to upload a Full Directory, please Zip it first (use WinZip)
    for fn in uploaded.keys():
        name = fn  #.encode('utf-8')
        #print('\nfile after encode', name)
        #name = io.BytesIO(uploaded[name])
    if case_f == 0:    # case of uploading 'One File only'
        print('\n file name: ', name)
        return name
    else:   # case of uploading a directory and its subdirectories and files
        zfile = zipfile.ZipFile(name, 'r')   # unzip the directory 
        zfile.extractall()
        for d in zfile.namelist():   # d = directory
            print('\n main directory name: ', d)
            return d
print('Done!')

Done!


In [0]:
file_name = upload_dir_file(0)


Saving Test4.csv to Test4.csv
Saving time_series.py to time_series.py
Saving Train4.csv to Train4.csv
('\n file name: ', u'Test4.csv')


In [0]:
import numpy as np
from time_series import *

Using TensorFlow backend.


In [0]:
training_file='Train4.csv'
test_file='Test4.csv'

In [0]:
npop=3
num_parents=2
max_epochs=2
ndim=3
max_lim=np.array([120,50,5])
min_lim=np.array([20,10,2])
pop=np.zeros((npop,ndim))
fit=np.zeros(npop)

In [0]:
params={'timestep':20,
        'dropout_prob':0.1,
        'num_units': 50,
        'num_hidden_layers':4,
        'mode': 'training'}

In [0]:
rnn=time_series_analysis(training_file,test_file)

In [0]:
pop=min_lim+np.random.rand(npop,ndim)*(max_lim-min_lim)

In [0]:
for i in range(npop):
    timestep,num_units,num_hidden_layers=pop[i,:]
    
    params['timestep']=int(round(timestep))
    params['num_hidden_layers']=int(round(num_hidden_layers))
    params['num_units']=int(round(num_units))
    
    fit[i]=np.divide(1,1+rnn.get_validation_error(params,1)) 

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/1
76/76 [==============================] - 4s 51ms/step - loss: 0.0354
Epoch 1/1
67/67 [==============================] - 2s 25ms/step - loss: 0.0547
Epoch 1/1
98/98 [==============================] - 2s 23ms/step - loss: 0.0498


In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
indices=np.argsort(-fit)
pop=pop[indices,:]
fit=fit[indices]

In [0]:
mu_prob=0.05

In [0]:
for g in range(max_epochs):
    
    #selection
    fit_p1=np.array([fit[0]])
    parent1=np.ndarray.tolist(pop[0,:])
    
    new_pop=pop[1:,:]
    new_fit=fit[1:]
    
    random_int=np.random.choice(npop-1,size=(num_parents-1),replace=False)
    
    parents=np.ndarray.tolist(new_pop[random_int,:])
    fit_p=new_fit[random_int]
    
    parents.append(parent1)
    fit_p=np.concatenate((fit_p,fit_p1))
    
    parents=np.array(parents)
    
    fit_child=np.array([])
    child_set=[]
    
    #crossover and mutation
    for i in range(num_parents):
        j=i+1
        
        while (j>i) and (j<num_parents):
            
            rho=np.random.rand()
            child1=rho*parents[i,:]+(1-rho)*parents[j,:]
            
            for l in range(len(child1)):
                child1[l]=np.amax([child1[l],min_lim[l]])
                child1[l]=np.amin([child1[l],max_lim[l]])
                
            timestep,dropout_prob,num_hidden_layers=child1     #num_units,num_hidden_layers=child1  <follow the same for every child>
            timestep=int(timestep)
            num_hidden_layers=int(num_hidden_layers)
            fit_c1=np.divide(1,1+rnn.get_validation_error(params,1))
            
            child2=rho*parents[j,:]+(1-rho)*parents[i,:]
            
            for l in range(len(child2)):
                child2[l]=np.amax([child2[l],min_lim[l]])
                child2[l]=np.amin([child2[l],max_lim[l]])
                
            timestep,dropout_prob,num_hidden_layers=child2
            timestep=int(timestep)
            num_hidden_layers=int(num_hidden_layers)
            fit_c2=np.divide(1,1+rnn.get_validation_error(params,1))
            
            if fit_c2>fit_c1:
                child=child2
                fit_c=fit_c2
            else:
                child=child1
                fit_c=fit_c1
                
            chi=np.random.rand()
            if mu_prob>chi:
                eta=np.random.choice(len(child),replace=False)
                zeta1=np.random.rand()
                zeta2=np.random.rand()
                child[eta]=(zeta1+zeta2)*child[eta]
                child[eta]=np.amax([child[eta],min_lim[eta]])
                child[eta]=np.amin([child[eta],max_lim[eta]])
                
                timestep,dropout_prob,num_hidden_layers=child
                timestep=int(timestep)
                num_hidden_layers=int(num_hidden_layers)
                fit_c=np.divide(1,1+rnn.get_validation_error(params,1))
            child=np.ndarray.tolist(child)
            child_set.append(child) 
            fit_child=np.concatenate((fit_child,np.array([fit_c])))
            j=j+1
    pop_new=np.ndarray.tolist(pop)
    for c in range(len(child_set)):
        pop_new.append(child_set[c])
    pop=np.array(pop_new)
    fit=np.concatenate((fit,fit_child))
    indices=np.argsort(-fit)
    pop=pop[indices,:]
    pop=pop[0:npop,:]
    fit=fit[indices]
    fit=fit[0:npop]

Epoch 1/1
98/98 [==============================] - 2s 25ms/step - loss: 0.0261
Epoch 1/1
98/98 [==============================] - 3s 27ms/step - loss: 0.0193
Epoch 1/1
98/98 [==============================] - 3s 30ms/step - loss: 0.0455
Epoch 1/1
98/98 [==============================] - 3s 33ms/step - loss: 0.0627
Epoch 1/1
98/98 [==============================] - 3s 35ms/step - loss: 0.0370


In [0]:
timestep_opt,num_units_opt,num_hidden_opt=pop[0,:]
params['timestep']=int(round(timestep))
params['num_hidden_layers']=int(round(num_hidden_layers))
params['num_units']=int(round(num_units))

print('The optimum timestep, number of units per layer and number of hidden layers are given respectively:',round(timestep),round(num_units),round(num_hidden_layers))

('The optimum timestep, number of units per layer and number of hidden layers are given respectively:', 85.0, 10.0, 3.0)


In [0]:
params['mode']='predict'
rnn.plot_stock_price(params,500)